<a href="https://colab.research.google.com/github/mnassar/py2vpy3xai/blob/main/py2vpy3XaiSlidingWindow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

@author: mnassar

Sliding window explainer.

This code was used to generate the results in Table 4:

Example of a mini-dataset for our sliding window
explainer

In [2]:
from keras import models
model_str = 'MM_dataset10k_Win5_NF32.keras'
# model_str = 'MM_dataset80k_Win5_NF64.keras'
# model_str = 'MM_dataset80k_Win5_NF128.keras'
# model_str = 'MM_dataset80k_Win7_NF32.keras'
# model_str = 'MM_dataset80k_Win7_NF64.keras'
# model_str = 'MM_dataset80k_Win7_NF128.keras'
# model_str = 'MM_dataset80k_Win10_NF32.keras'
# model_str = 'MM_dataset80k_Win10_NF64.keras'
# model_str = 'MM_dataset80k_Win10_NF128.keras'
m = models.load_model(model_str)
MAX_LEN = 100
WIN_SIZE = int (model_str.split("_")[2][3:])
EXPL_WIN = WIN_SIZE + 3
EXPL_EVAL_THR = 3
dataset = model_str.split("_")[1]+".csv"
m.summary()

Model: "MM_dataset10k_Win5_NF32"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 100)]                0         []                            
                                                                                                  
 embedding (Embedding)       (None, 100, 8)               2048      ['input_1[0][0]']             
                                                                                                  
 conv1d_1 (Conv1D)           (None, 100, 32)              1312      ['embedding[0][0]']           
                                                                                                  
 conv1d (Conv1D)             (None, 100, 32)              1312      ['embedding[0][0]']           
                                                                            

In [3]:
print(WIN_SIZE)
print(dataset)

5
dataset10k.csv


In [4]:
from keras.utils import pad_sequences

# sample to explain
line = "from __future__ import print_function"
def predict (lines):
  samples = [  bytearray(line, encoding="utf-8") for line in lines ]
  samples_padded = pad_sequences(samples, maxlen=MAX_LEN, padding='post', truncating='post')
  preds = m.predict(samples_padded)
  # if pred < 0.5:
  #   label="python2"
  # else:
  #   label="python3"
  return preds

predict([line, line])

1/1 [==============================] - 0s 318ms/step


array([[0.04795397],
       [0.04795397]], dtype=float32)

In [5]:
import pandas as pd
# sample neighboring
def expl(loc):
  lines = []
  for i in range (len(loc)):
    line = loc[:i] + " " * WIN_SIZE + loc [i+WIN_SIZE:]
    print(line)
    lines.append (line)
  lines.append(loc)

  y_train = predict (lines)
  y_ref = y_train[-1][0]
  print(y_ref)
  df_t = pd.DataFrame(data=enumerate(lines[:-1]), columns=["index", "loc"])
  df_t["pred"] = y_train[:-1]
  df_t["gap"] = df_t["pred"].map(lambda y: abs(y - y_ref))
  print (df_t)
  print (df_t["gap"].idxmax())
  idx = df_t.loc[df_t["gap"].idxmax()]["index"]
  print (idx)
  start_idx = max(idx - 2, 0)
  return line[start_idx:start_idx+WIN_SIZE+3]


expl = expl(line)
expl


     __future__ import print_function
f     _future__ import print_function
fr     future__ import print_function
fro     uture__ import print_function
from     ture__ import print_function
from      ure__ import print_function
from _     re__ import print_function
from __     e__ import print_function
from __f     __ import print_function
from __fu     _ import print_function
from __fut      import print_function
from __futu     import print_function
from __futur     mport print_function
from __future     port print_function
from __future_     ort print_function
from __future__     rt print_function
from __future__      t print_function
from __future__ i      print_function
from __future__ im     print_function
from __future__ imp     rint_function
from __future__ impo     int_function
from __future__ impor     nt_function
from __future__ import     t_function
from __future__ import      _function
from __future__ import p     function
from __future__ import pr     unction
from __futur

'__future'

Evaluate sliding window explanations

In [7]:
# prepare data once for all
df = pd.read_csv(dataset, dtype={
    '__future__': 'object',
    'xrange': 'object'})
df = df.drop ( df[df["lines of code"].isnull()].index )
# df

In [8]:
# consider only lines with annotated explanations
df_expl = df[df["explanation"].notnull()]
# add predictions
df_expl = df_expl.join( pd.Series(name="pred", data = predict(df_expl["lines of code"]).reshape(-1), index=df_expl.index) )
# consider only correct predictions
df_expl["pred_class"] = df_expl["pred"].map(lambda x: 3 if x>0.5 else 2)
df_expl = df_expl.loc[df_expl["pred_class"] == df_expl["class"]]
df_expl.shape


59/59 [==============================] - 0s 4ms/step


(1859, 14)

In [9]:
def expl_extract(r):
  start_idx = max(r["idx_expl"] - 2, 0)
  line = r["lines of code"]
  return line[start_idx:start_idx + WIN_SIZE + 3]

In [10]:
# evaluate explainability
import re
EXPL_EVAL_THR = 3

#longest common subsequence
def lcs(X, Y):
    # find the length of the strings
    m = len(X)
    n = len(Y)
    # declaring the array for storing the dp values
    L = [[None]*(n + 1) for i in range(m + 1)]
    """Following steps build L[m + 1][n + 1] in bottom up fashion
    Note: L[i][j] contains length of LCS of X[0..i-1]
    and Y[0..j-1]"""
    for i in range(m+1):
        for j in range(n+1):
            if i == 0 or j == 0 :
                L[i][j] = 0
            elif X[i-1] == Y[j-1]:
                L[i][j] = L[i-1][j-1]+1
            else:
                L[i][j] = 0
    return max ( max(l) for l in L )

def eval_expl(r, col):
  if re.search(re.compile(r'\bu[\'|\"]\s*\S+'), r[col]):
    return True
  return lcs(r["explanation"], r[col]) >= EXPL_EVAL_THR

In [11]:

def expl_all(locs):
  df_x = pd.DataFrame(columns=["index", "loc", "ref"], \
    data = [ [i, loc[:i] + " " * WIN_SIZE + loc [i+WIN_SIZE:], l] \
            for l, loc in enumerate(locs) \
             for i in range (len(loc)) \
              ] )
  print (df_x.shape)
  y_train = predict(df_x["loc"])
  y_ref = predict(locs)
  df_x["pred"] = list( y_train.reshape(-1) )
  df_x["gap"] = df_x.apply(lambda r: abs(r["pred"] - y_ref[r["ref"]][0]), axis=1)

  return df_x


In [13]:
%%time
# sum_acc = 0
# for i in range(0, df_expl.shape[0], 10000):
# df_small = df_expl.iloc[i:i+10000].copy()
df_small = df_expl
df_t = expl_all(df_small["lines of code"])
maxidx = df_t.groupby("ref")['gap'].idxmax()
df_small["idx_expl"]=df_t.loc[maxidx]["index"].values
df_small["slidingW"]=df_small.apply(expl_extract, axis=1)
df_small["acc_expl_slidingW"] = df_small.apply(eval_expl, col="slidingW", axis=1)
acc = sum(df_small["acc_expl_slidingW"])
print (f"{acc}/{df_small.shape[0]}")
# sum_acc += acc

(83889, 3)
59/59 [==============================] - 0s 3ms/step
1514/1859
CPU times: user 14.1 s, sys: 706 ms, total: 14.8 s
Wall time: 12.9 s


In [14]:
f"{acc * 100 / df_expl.shape[0]:.2f}"
# f"{sum_acc * 100 / df_expl.shape[0]:.2f}"

'81.44'

In [ ]:
# type(df_t.loc[maxidx]["index"])
# df_small["idx_expl"].isnull().any()

In [ ]:
# I am not looking for the patterns for the moment
# for col in ['print(', '__future__', 'xrange', ' range',
#             'u"', 'print ', 'unicode(', '__next__(', 'raw_input(']:
#   den = df_small [ df_small[col].notnull()].shape[0]
#   num = df_small [ df_small[col].notnull()].loc[ df_small["acc_expl_slidingW"] == True ].shape[0]
#   print (col, den, num)

In [ ]:
# The old way, much slower
# can be used for verification (the two methods must give equal values)

# df_small = df_expl.head(100).copy()
# df_small["slidingW"] = df_small["lines of code"].map(slidingW)
# df_small["acc_expl_slidingW"] = df_small.apply(eval_expl, col="slidingW", axis=1)
# exp_acc_max = sum(df_small["acc_expl_slidingW"]) * 100 / df_small.shape[0]
# f"{exp_acc_max:.2f}"